In [1]:
from datetime import datetime
cur_date_time = datetime.today().strftime('%Y_%m_%d_%H_%M')
log_dir = "../../logs/"
project_name = 'textattack_lib_textfooler_WHOLE_SCIFACT_refute_major_checklist'
version = '0.0.1'
log_file_dir_name = log_dir+project_name+'_'+version+'.log'

import logging
  
#Create and configure logger
logging.basicConfig(filename=log_file_dir_name,
                    level=logging.INFO,
                    format='%(message)s',
                    filemode='w')

log_file_fine_tune_callback = '../../logs/log_textattack_lib_refute_major_checklist.txt'

In [2]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

In [3]:
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack
from textattack.transformations import WordSwap

from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset
import textattack

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.utils.data import Dataset, DataLoader
#import pytorch_lightning as pl

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM

from sentence_transformers import SentenceTransformer, util
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup

import re
import sys
import random
from random import choices
import os
from itertools import chain
from string import punctuation
import time
import argparse

import csv
import json

import logging

import traceback
import gc
from enum import Enum 

import numpy as np
import pandas as pd
from statistics import mean

import matplotlib.pyplot as plt


# from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
# from trl.ppo import PPOTrainer
#from trl.core import build_bert_batch_from_txt

from IPython.core.display import Markdown,display, HTML, Latex
import qgrid

from verisci.covid import AbstractRetriever, RationaleSelector, LabelPredictor
from verisci.evaluate.lib.data import GoldDataset

from GPUtil import showUtilization as gpu_usage
import wandb

from tqdm.notebook import tqdm
tqdm.pandas()
import pickle

In [5]:
from sklearn import preprocessing


In [6]:
pd.set_option("display.precision", 2)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
class ParaphraseTargetDirection(Enum):
    org_support_to_gen_refute = 0
    org_refute_to_gen_support = 1
    
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(37)    

In [9]:
logging.info("#### \n\n>>> Starting... >>> ")

In [10]:
PARAPHRASE_PROJECT_SETTINGS={
    'file_and_dirs': {
        'file_paraphrased_no_tune_all_model_full' : '../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl', # can be deleted
        'file_org_claims_by_scifact' : '../../dfs_generated/scifact/org_claim_ext_label_roberta_large_fever.pkl',
    },
    'config_scifact' : {
        'cls_model_name': '../../scifact/model/label_roberta_large_fever_scifact',
        'rationale_model_name': '../../scifact/model/rationale_roberta_large_fever_scifact',
        'loc_gold_ds_corpus' : '../../scifact/data/corpus.jsonl', 
        'loc_gold_ds_train' : '../../scifact/data/claims_train.jsonl', 
        'loc_gold_ds_dev' : '../../scifact/data/claims_dev.jsonl', 

    },
    
    
    'paraphrase_model' :
    {
        'list_potential_paraphrase_models' : 
            [
                {'model_name' : 'parasci_base_no_fine_tune' , 'model_path_or_url' : 'HelloRusk/t5-base-parasci', 'is_selected' : False},
                {'model_name' : 'parrot_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'parrot_div_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'pegasus_base_no_fine_tune' , 'model_path_or_url' : 'tuner007/pegasus_paraphrase', 'is_selected' : False},
                {'model_name' : 'paws_base_no_fine_tune' , 'model_path_or_url' : 'Vamsi/T5_Paraphrase_Paws', 'is_selected' : True},
                {'model_name' : 'tapaco_base_no_fine_tune' , 'model_path_or_url' : 'hetpandya/t5-base-tapaco', 'is_selected' : False},
                {'model_name' : 'sci_five_pubmed' , 'model_path_or_url' : 'razent/SciFive-large-Pubmed_PMC', 'is_selected' : False}
            ],
        't5_paraphrase_model_params':
        {
            'max_length':256,
            'do_sample':True,
            'top_k':50,
            'top_p': 0.99,
            'repetition_penalty':3.5,
            'early_stopping':True,
            'num_return_sequences':20
        }
    },
    
    'run_settings':
    {
        'PARAPHRASE_FT_TRAIN_SPLIT' : 0.1,
        'PARAPHRASE_FT_DATASET_DIRECTION' : ParaphraseTargetDirection.org_refute_to_gen_support,#ParaphraseTargetDirection.org_support_to_gen_refute,#ParaphraseTargetDirection.org_refute_to_gen_support,
        'NUM_OF_EPOCH_REQ_FT' : 2,
        'FILTER_BY' : 'TECH_TERMS',
        'SIMILARITY_THRESHOLD' : -100
        #'CUR_MODEL_NAME_PATHS' : (lambda: [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True])(),
    },
}

CUR_NO_OF_EPOCH_FT = 0

## Scifact Functinos

### Load data

In [11]:
def get_claim_label_from_jsonl(dataset_jsonl):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name

            tmp_dic = {"claim" : claim_txt, "label" : claim_label}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [12]:
def get_claim_label_evidence_from_jsonl(dataset_jsonl, source):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = claim_train.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name
            
            list_rationales = []
            for i, sents in enumerate(evidence.rationales):
                list_rationales = [sent for i, sent in enumerate(ev_doc.sentences) if i in sents]

            tmp_dic = {"claim" : claim_txt, "label" : claim_label, "list_rationales" :list_rationales, "source" :source}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [13]:
ds_train = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_train'])
claim_train = ds_train.get_claim(39)
claim_train.pretty_print()

dic_train = get_claim_label_evidence_from_jsonl(ds_train, source = "train")

Example 39: A diminished ovarian reserve does not solely indicate infertility in an a priori non-infertile population.

Evidence sets:

####################

13497630: SUPPORTS
Set 0:
	- After adjusting for age, body mass index, race, current smoking status, and recent hormonal contraceptive use, women with low AMH values (<0.7 ng/mL [n = 84]) did not have a significantly different predicted probability of conceiving by 6 cycles of attempt (65%; 95% CI, 50%-75%) compared with women (n = 579) with normal values (62%; 95% CI, 57%-66%) or by 12 cycles of attempt (84% [95% CI, 70%-91%] vs 75% [95% CI, 70%-79%], respectively).
Set 1:
	- Women with high serum FSH values (>10 mIU/mL [n = 83]) did not have a significantly different predicted probability of conceiving after 6 cycles of attempt (63%; 95% CI, 50%-73%) compared with women (n = 654) with normal values (62%; 95% CI, 57%-66%) or after 12 cycles of attempt (82% [95% CI, 70%-89%] vs 75% [95% CI, 70%-78%], respectively).
Set 2:
	- Women

In [14]:
ds_valid = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_dev'])
claim_valid = ds_valid.get_claim(42)
claim_valid.pretty_print()

dic_valid = get_claim_label_evidence_from_jsonl(ds_valid, source = "dev")

Example 42: A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.

Evidence sets:

####################

18174210: REFUTES
Set 0:
	- Individuals homozygous for alpha(+)-thalassaemia have microcytosis and an increased erythrocyte count.
	- We estimated that the haematological profile in children homozygous for alpha(+)-thalassaemia reduces the risk of SMA during acute malaria compared to children of normal genotype (relative risk 0.52; 95% confidence interval [CI] 0.24-1.12, p = 0.09).   

Set 1:
	- CONCLUSIONS The increased erythrocyte count and microcytosis in children homozygous for alpha(+)-thalassaemia may contribute substantially to their protection against SMA.


In [15]:
def get_dataframes_by_majority_org_claim(df_all_paraphrased_org_claim):
    df_all_paraphrased_org_success = df_all_paraphrased_org_claim[df_all_paraphrased_org_claim['org_comment'] == 'success']
    
    # Select claims with majority
    df_paraphrased_org_support_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] > df_all_paraphrased_org_success['org_count_refute']
    ]

    
    df_paraphrased_org_refute_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] < df_all_paraphrased_org_success['org_count_refute']
    ]
    
    return df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_org_success

In [16]:
#

df_org_claims_by_scifact = pd.read_pickle(PARAPHRASE_PROJECT_SETTINGS['file_and_dirs']['file_org_claims_by_scifact'])
df_org_claims_by_scifact_support_major, df_org_claims_by_scifact_refute_major, df_org_claims_by_scifact_sci_success = get_dataframes_by_majority_org_claim(df_org_claims_by_scifact)
df_org_claims_by_scifact_support_major['label'] = 'support'
df_org_claims_by_scifact_refute_major['label'] = 'refute'
df_org_claims_by_scifact_majority = pd.concat([df_org_claims_by_scifact_support_major, df_org_claims_by_scifact_refute_major], ignore_index=True)

/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
#df_claim_evid_label = df_org_claims_by_scifact.copy()
#df_claim_evid_label = df_org_claims_by_scifact_support_major.copy()#df_org_claims_by_scifact_majority.copy()
df_claim_evid_label = df_org_claims_by_scifact_refute_major.copy()

In [18]:
df_claim_evid_label

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,org_list_supported_confidence_mean,org_list_refuted_confidence_mean,org_comment,label
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389873027801514],0.00,0.64,success,refute
17,ART has no effect on the infectiveness of HIV-...,REFUTES,[CONCLUSIONS Mathematical models evaluating th...,train,0,11,[],"[3vynbdli.00002, bhzgumb8, qj7lho9r, 3c8phatx....",[],"[0.8876776695251465, 0.7819303870201111, 0.765...",0.00,0.67,success,refute
33,Ambulatory blood pressure monitoring is inaccu...,REFUTES,[Ambulatory monitoring before the start of lif...,train,0,7,[],"[ihk16l1k, 9fbbwrte, lkpeoiyp, b4qq72r3, k7ce1...",[],"[0.8934949636459351, 0.6645402908325195, 0.644...",0.00,0.65,success,refute
40,Antidepressants increase the severity of migra...,REFUTES,[CONCLUSIONS Tricyclic antidepressants are eff...,train,1,2,[brytb50b],"[oyb8dczr, kzy33hdb.00128]",[0.946519136428833],"[0.6452325582504272, 0.6240962743759155]",0.95,0.63,success,refute
42,Antimicrobial agents are less effective due to...,SUPPORTS,[Overusing antibiotics is not the only cause a...,train,6,9,"[tfr2xbxu.00064, 8wvclg4o.00006, ytv7dwr0.0004...","[8dxv04uu.00010, bwn4bmf5.00017, pzjj2wxc.0000...","[0.9899699091911316, 0.7421004772186279, 0.677...","[0.9373342394828796, 0.8747098445892334, 0.837...",0.70,0.71,success,refute
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,There is an inverse relationship between hip f...,SUPPORTS,[CONCLUSIONS These findings support an associa...,dev,0,5,[],"[f1br2h6p.00045, 7r35ow4y.00059, 46oo3sse.0000...",[],"[0.911043107509613, 0.7290191054344177, 0.6985...",0.00,0.70,success,refute
676,Thigh-length graduated compression stockings (...,SUPPORTS,[INTERPRETATION These data do not lend support...,dev,3,4,"[37zrbye6, ie9dn724.00008, 95yiliim]","[nuadau49, 2uzai25k, 2n9f82xs, 6vxtpn90]","[0.8855438232421875, 0.7021892070770264, 0.672...","[0.6754951477050781, 0.637149453163147, 0.5785...",0.75,0.61,success,refute
677,Tirasemtiv has no effect on fast-twitch muscle.,REFUTES,[Binding selectively to the fast-skeletal-trop...,dev,2,4,"[kxfsusgo, 2v4w7ail]","[jns9aaez, h2p8id96, fkqkbail, 16skmh6b.00030]","[0.8443883061408997, 0.7982557415962219]","[0.9675067067146301, 0.9525337815284729, 0.847...",0.82,0.87,success,refute
679,Transplanted human glial progenitor cells are ...,REFUTES,[Long-term potentiation (LTP) was sharply enha...,dev,0,1,[],[dkze9ees.00047],[],[0.548649787902832],0.00,0.55,success,refute


### Scifact Model

In [19]:
class ArgsScifact:
    def __init__(self, claim):
        self.claim = claim
        self.report_file = "../../scifact/results/covid/report" #not needed
        self.n_documents = 100
        self.rationale_selection_method = "topk"
        self.output_format = "markdown"
        self.rationale_threshold = 0.5
        self.label_threshold = 0.5
        self.keep_nei = False
        self.full_abstract = True
        self.verbose = True
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        ##
class PretrainedModelsForScifact:
    def __init__(self, args):
        if args.device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(args.device)
            
        #self.rationale_selection_model = '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_scifact'
        self.rationale_selection_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']
        self.label_prediction_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name']
        self.abstract_retriever = AbstractRetriever()
        self.rationale_selector = RationaleSelector(self.rationale_selection_model,
                                               args.rationale_selection_method,
                                               args.rationale_threshold,
                                               self.device)
        self.label_predictor = LabelPredictor(self.label_prediction_model,
                                         args.keep_nei,
                                         args.label_threshold,
                                         self.device)


In [20]:
args_sci = ArgsScifact("")

pretrained_models_config = PretrainedModelsForScifact(args_sci)

Some weights of the model checkpoint at ../../scifact/model/rationale_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ../../scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSe

In [21]:
log_failed_claim = []
def inference(args, pretraind_models_config):

    try:
#         if args.verbose:
#         print("Retrieving abstracts. inference > ", args.claim)
        results = pretraind_models_config.abstract_retriever(args.claim, k=args.n_documents)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'abstract retrival', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        #print("abstract_retriever >> ", results)

#         if args.verbose:
#             print("Selecting rationales. inference > ", args.claim)
        results = pretraind_models_config.rationale_selector(args.claim, results)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'Rationale selection', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
#         if args.verbose:
#             print("Label predictions. inference > ", args.claim)
        results = pretraind_models_config.label_predictor(args.claim, results)

        if len(results) == 0:
            temp_dic = {'failed_in' : 'Label Prediction', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        results.sort(key=lambda r: r['label_confidence'], reverse=True)
        return results
    except Exception as e:
        print("Exception :: Inference cant retrive info for >> ", args.claim)
        print(sys.exc_info()[0])
        print(traceback.format_exc())
        temp_dic = {'failed_in' : sys.exc_info()[0], 'claim': args.claim}
        log_failed_claim.append(temp_dic)
        return []


In [22]:
def write_result(result, full_abstract):
    all_msg = ""
    all_msg = f"#### [{result['title']}]({result['url']}) \n"
    #print(msg, file=f)
    #all_msg = all_msg+msg
    ev_scores = [f"{x:0.2f}" for x in result["evidence_confidence"]]
    ev_scores = ", ".join(ev_scores)
    if result['label'].lower() == "support":
        msg = f"🟩 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    elif result['label'].lower() == "refute":
        msg = f"🟥 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    else:
        msg = f"⏺ **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    #print(msg, file=f)
    all_msg = all_msg+msg 
    
    for i, line in enumerate(result["abstract"]):
        # If we're showing the full abstract, show evidence in green.
        if full_abstract:
            if result['label'].lower() == "support":
                msg = (f"- <span style='color:green'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")
            elif result['label'].lower() == "refute":
                msg = (f"- <span style='color:red'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")                
            #print(msg, file=f)
            all_msg = all_msg+msg + " \n"
        else:
            if i in result["evidence"]:
                msg = f"- {line}"
                #print(msg, file=f)
                all_msg = all_msg+msg + " \n" 
    
    #print(file=f)
    #print(40 * "-", file=f)
    #print(file=f)
    all_msg = all_msg+msg 
    return all_msg + "\n"

In [23]:
def export(args, results):
    all_msg = ""
    claim = args.claim
    #report_file = args.report_file
    #f = open(f"{report_file}.md", "w")
    msg = f"### Claim \n > **{claim}** \n "
    #print(msg, file=f)
    #print(file=f)
    all_msg = all_msg +msg
    
    #support_confs = [], refute_confs = []
    confs = []
    for result in results:
        if result['label'].lower() == "support":
            tmp_dic = {'label' : 'Support', 'label_confidence' : result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        elif result['label'].lower() == "refute":
            tmp_dic = {'label' : 'Refute', 'label_confidence' : -result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        
    
    tpm_df = pd.DataFrame(confs)
    #HTML(tpm_df.style.bar(align='mid', color=['#d65f5f', '#5fba7d']))
    display(HTML(tpm_df.style.bar(subset=["label_confidence"], align='mid', color=['#ffa1a1', '#bfffcf']).render()))
    
    msg = "### Evidence \n "
    all_msg = all_msg +msg
    for result in results:
        cur_msg = write_result(result, args.full_abstract)
        all_msg = all_msg +cur_msg+"\n"

    return all_msg

In [24]:
claim_to_check = "ART substantially reduces infectiveness of HIV-positive people."#df_claim_evid_label.iloc[18, :]["claim"]
args_sci = ArgsScifact(claim_to_check)

#pretrained_models_config = pretrained_models_for_scifact(args_sci)

results_raw = inference(args_sci, pretrained_models_config)

if results_raw!= []:
    result_md = export(args_sci, results_raw)
    #result_md = export(args_sci, results_raw)
    display(Markdown(result_md))

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


,label,label_confidence,no_of_evidence
0,Support,0.69,3
1,Support,0.64,3
2,Support,0.54,3


### Claim 
 > **ART substantially reduces infectiveness of HIV-positive people.** 
 ### Evidence 
 #### [Autonomous Targeting of Infectious Superspreaders Using Engineered Transmissible Therapies](https://api.semanticscholar.org/10.1371/journal.pcbi.1002015) 
🟩 **Decision** : SUPPORT (score=0.69, evidence scores=0.11, 0.06, 0.01)
- Infectious disease treatments, both pharmaceutical and vaccine, face three universal challenges: the difficulty of targeting treatments to high-risk ‘superspreader’ populations who drive the great majority of disease spread, behavioral barriers in the host population (such as poor compliance and risk disinhibition), and the evolution of pathogen resistance. 
- Here, we describe a proposed intervention that would overcome these challenges by capitalizing upon Therapeutic Interfering Particles (TIPs) that are engineered to replicate conditionally in the presence of the pathogen and spread between individuals — analogous to ‘transmissible immunization’ that occurs with live-attenuated vaccines (but without the potential for reversion to virulence). 
- Building on analyses of HIV field data from sub-Saharan Africa, we construct a multi-scale model, beginning at the single-cell level, to predict the effect of TIPs on individual patient viral loads and ultimately population-level disease prevalence. 
- <span style='color:green'>Our results show that a TIP, engineered with properties based on a recent HIV gene-therapy trial, could stably lower HIV/AIDS prevalence by ∼30-fold within 50 years and could complement current therapies.</span> 
- <span style='color:green'>In contrast, optimistic antiretroviral therapy or vaccination campaigns alone could only lower HIV/AIDS prevalence by <2-fold over 50 years.</span> 
- The TIP's efficacy arises from its exploitation of the same risk factors as the pathogen, allowing it to autonomously penetrate superspreader populations, maintain efficacy despite behavioral disinhibition, and limit viral resistance. 
- <span style='color:green'>While demonstrated here for HIV, the TIP concept could apply broadly to many viral infectious diseases and would represent a new paradigm for disease control, away from pathogen eradication but toward robust disease suppression.</span> 
- <span style='color:green'>While demonstrated here for HIV, the TIP concept could apply broadly to many viral infectious diseases and would represent a new paradigm for disease control, away from pathogen eradication but toward robust disease suppression.</span>

#### [HIV: Biology to Treatment](https://api.semanticscholar.org/10.1007/978-981-32-9898-9_7) 
🟩 **Decision** : SUPPORT (score=0.64, evidence scores=0.67, 0.01, 0.37)
- AIDS is one of the most dreaded diseases of the twenty-first century caused by human immunodeficiency virus (HIV). 
- <span style='color:green'>Recently, there are reports which show decline in new infections due to better access to anti-retroviral drugs.</span> 
- <span style='color:green'>Still on a daily basis, ~2356 new HIV infections are being reported globally.</span> 
- New treatments and anti-HIV drugs are being continuously developed with the aim to control and cure AIDS. 
- The anti-HIV drugs that are in use usually target HIV entry and replication inside the host cells. 
- <span style='color:green'>However, these drugs are only partially effective in slowing the rate of HIV replication.</span> 
- Nevertheless, the virus manages to replicate at much slower rates even when anti-retroviral treatment is ongoing. 
- The HIV seropositives who are on anti-retroviral treatment for long periods of time are now developing different kinds of other complications including neuroAIDS. 
- The latest development in HIV therapy is a novel kind of bone marrow transplantation from donors who have a homozygous mutation in CCR5 gene. 
- The latest development in HIV therapy is a novel kind of bone marrow transplantation from donors who have a homozygous mutation in CCR5 gene.

#### [Human Immunodeficiency Virus-Associated Diarrhea: Still an Issue in the Era of Antiretroviral Therapy](https://api.semanticscholar.org/10.1007/s10620-015-3615-y) 
🟩 **Decision** : SUPPORT (score=0.54, evidence scores=0.01, 0.99, 0.02)
- <span style='color:green'>Over half of patients with human immunodeficiency virus (HIV) experience diarrhea that contributes negatively to quality of life and adherence to antiretroviral therapy (ART).</span> 
- Opportunistic infectious agents that cause diarrhea in patients with HIV span the array of protozoa, fungi, viruses, and bacteria. 
- <span style='color:green'>With global use of ART, the incidence of diarrhea because of opportunistic infections has decreased; however, the incidence of noninfectious diarrhea has increased.</span> 
- <span style='color:green'>The etiology of noninfectious diarrhea in patients with HIV is multifactorial and includes ART-associated diarrhea and gastrointestinal damage related to HIV infection (i.e., HIV enteropathy).</span> 
- A basic algorithm for the diagnosis of diarrhea in patients with HIV includes physical examination, a review of medical history, assessment of HIV viral load and CD4+ T cell count, stool microbiologic assessment, and endoscopic evaluation, if needed. 
- For patients with negative diagnostic results, the diagnosis of noninfectious diarrhea may be considered. 
- Pharmacologic options for the treatment of noninfectious diarrhea are primarily supportive; however, the use of many unapproved agents is based on unstudied and anecdotal information. 
- In addition, these agents can be associated with treatment-limiting adverse events (AEs), such as drug–drug interactions with ART regimens, abuse liability, and additional gastrointestinal AEs. 
- Currently, crofelemer, an antisecretory agent, is the only therapy approved in the USA for the symptomatic relief of noninfectious diarrhea in patients with HIV on ART. 
- Currently, crofelemer, an antisecretory agent, is the only therapy approved in the USA for the symptomatic relief of noninfectious diarrhea in patients with HIV on ART.



In [25]:
#print(pretraind_models_config)

## Scifact Extended funtions


In [26]:
def get_prob_score_claim(claim_to_analyze):
    logging.info("#### \n\n>>> Claim >>> ")
    logging.info(claim_to_analyze)

    args_sci = ArgsScifact(claim_to_analyze)
    dic_info = {}
    
    dic_info["org_count_support"] = 0
    dic_info["org_count_refute"] = 0
    dic_info["org_list_supported_ids"] = []
    dic_info["org_list_refuted_ids"] = []
    dic_info["org_list_supported_confidence"] = []
    dic_info["org_list_refuted_confidence"] = []
    dic_info["org_list_supported_confidence_mean"] = 0.0
    dic_info["org_list_refuted_confidence_mean"] = 0.0
    dic_info["org_comment"] = ""
    
    try:
        results_raw_org = inference(args_sci, pretrained_models_config)  
        
        if results_raw_org == []:
            dic_info["org_comment"] = "no result"
            
        else:
            list_supported_ids = [cur_result['id'] for cur_result in results_raw_org if cur_result['label'] == 'SUPPORT']
            list_refuted_ids = [cur_result['id'] for cur_result in results_raw_org if cur_result['label'] == 'REFUTE']
            list_supported_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_org if cur_result['label'] == 'SUPPORT']
            list_refuted_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_org if cur_result['label'] == 'REFUTE']
            
            
            dic_info["org_count_support"] = len(list_supported_ids)
            dic_info["org_count_refute"] = len(list_refuted_ids)
            dic_info["org_list_supported_ids"] = list_supported_ids
            dic_info["org_list_refuted_ids"] = list_refuted_ids
            dic_info["org_list_supported_confidence"] = list_supported_label_confidence
            dic_info["org_list_refuted_confidence"] = list_refuted_label_confidence
            if len(list_supported_label_confidence) > 0:
                dic_info["org_list_supported_confidence_mean"] = mean(list_supported_label_confidence)
            if len(list_refuted_label_confidence) > 0:
                dic_info["org_list_refuted_confidence_mean"] = mean(list_refuted_label_confidence)
            dic_info["org_comment"] = "success"
            
    except Exception as e:
        dic_info["org_comment"] = "exception : "+e
        logging.info(">>> Exception original claim >>> ") 
        logging.info(claim_to_analyze) 
        logging.info(e)

    finally:
        log_claim_outcome = '#claim_coutcome : '+claim_to_analyze + '||{support_count} : '+str(dic_info["org_count_support"]) + ' {refute_count} : '+str(dic_info["org_count_refute"])
        logging.info(log_claim_outcome)
        if ((dic_info["org_comment"] == "no result") | 
            (dic_info["org_count_support"] <= dic_info["org_count_refute"])):
            return np.array([1.0, 0.0])        
#         if ((dic_info["org_comment"] == "no result") | 
#             (dic_info["org_count_support"] >= dic_info["org_count_refute"])):
#             return np.array([1.0, 0.0])
        else:
            return np.array([0.0,1.0]) 
            #prob_support_majority = dic_info["org_count_support"]/(dic_info["org_count_support"] + dic_info["org_count_refute"])
            #return np.array([0.0,1.0])        
        
#     finally:
#         if dic_info["org_comment"] == "no result":
#             return np.array([1.0,0.0,0.0,0.0])
#         elif dic_info["org_count_support"] == dic_info["org_count_refute"]:
#             return np.array([0.0,1.0,0.0,0.0])
#         else:
#             prob_support_majority = dic_info["org_count_support"]/(dic_info["org_count_support"] + dic_info["org_count_refute"])
#             return np.array([0.0,0.0,prob_support_majority,(1-prob_support_majority)])

## Data Prep


In [27]:
df_claim_evid_label

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,org_list_supported_confidence_mean,org_list_refuted_confidence_mean,org_comment,label
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389873027801514],0.00,0.64,success,refute
17,ART has no effect on the infectiveness of HIV-...,REFUTES,[CONCLUSIONS Mathematical models evaluating th...,train,0,11,[],"[3vynbdli.00002, bhzgumb8, qj7lho9r, 3c8phatx....",[],"[0.8876776695251465, 0.7819303870201111, 0.765...",0.00,0.67,success,refute
33,Ambulatory blood pressure monitoring is inaccu...,REFUTES,[Ambulatory monitoring before the start of lif...,train,0,7,[],"[ihk16l1k, 9fbbwrte, lkpeoiyp, b4qq72r3, k7ce1...",[],"[0.8934949636459351, 0.6645402908325195, 0.644...",0.00,0.65,success,refute
40,Antidepressants increase the severity of migra...,REFUTES,[CONCLUSIONS Tricyclic antidepressants are eff...,train,1,2,[brytb50b],"[oyb8dczr, kzy33hdb.00128]",[0.946519136428833],"[0.6452325582504272, 0.6240962743759155]",0.95,0.63,success,refute
42,Antimicrobial agents are less effective due to...,SUPPORTS,[Overusing antibiotics is not the only cause a...,train,6,9,"[tfr2xbxu.00064, 8wvclg4o.00006, ytv7dwr0.0004...","[8dxv04uu.00010, bwn4bmf5.00017, pzjj2wxc.0000...","[0.9899699091911316, 0.7421004772186279, 0.677...","[0.9373342394828796, 0.8747098445892334, 0.837...",0.70,0.71,success,refute
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,There is an inverse relationship between hip f...,SUPPORTS,[CONCLUSIONS These findings support an associa...,dev,0,5,[],"[f1br2h6p.00045, 7r35ow4y.00059, 46oo3sse.0000...",[],"[0.911043107509613, 0.7290191054344177, 0.6985...",0.00,0.70,success,refute
676,Thigh-length graduated compression stockings (...,SUPPORTS,[INTERPRETATION These data do not lend support...,dev,3,4,"[37zrbye6, ie9dn724.00008, 95yiliim]","[nuadau49, 2uzai25k, 2n9f82xs, 6vxtpn90]","[0.8855438232421875, 0.7021892070770264, 0.672...","[0.6754951477050781, 0.637149453163147, 0.5785...",0.75,0.61,success,refute
677,Tirasemtiv has no effect on fast-twitch muscle.,REFUTES,[Binding selectively to the fast-skeletal-trop...,dev,2,4,"[kxfsusgo, 2v4w7ail]","[jns9aaez, h2p8id96, fkqkbail, 16skmh6b.00030]","[0.8443883061408997, 0.7982557415962219]","[0.9675067067146301, 0.9525337815284729, 0.847...",0.82,0.87,success,refute
679,Transplanted human glial progenitor cells are ...,REFUTES,[Long-term potentiation (LTP) was sharply enha...,dev,0,1,[],[dkze9ees.00047],[],[0.548649787902832],0.00,0.55,success,refute


In [28]:
le_ground = preprocessing.LabelEncoder()
le_ground.fit(df_claim_evid_label.label)
df_claim_evid_label['label_enc'] = le_ground.transform(df_claim_evid_label.label)

In [29]:
df_claim_evid_label

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,org_list_supported_confidence_mean,org_list_refuted_confidence_mean,org_comment,label,label_enc
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389873027801514],0.00,0.64,success,refute,0
17,ART has no effect on the infectiveness of HIV-...,REFUTES,[CONCLUSIONS Mathematical models evaluating th...,train,0,11,[],"[3vynbdli.00002, bhzgumb8, qj7lho9r, 3c8phatx....",[],"[0.8876776695251465, 0.7819303870201111, 0.765...",0.00,0.67,success,refute,0
33,Ambulatory blood pressure monitoring is inaccu...,REFUTES,[Ambulatory monitoring before the start of lif...,train,0,7,[],"[ihk16l1k, 9fbbwrte, lkpeoiyp, b4qq72r3, k7ce1...",[],"[0.8934949636459351, 0.6645402908325195, 0.644...",0.00,0.65,success,refute,0
40,Antidepressants increase the severity of migra...,REFUTES,[CONCLUSIONS Tricyclic antidepressants are eff...,train,1,2,[brytb50b],"[oyb8dczr, kzy33hdb.00128]",[0.946519136428833],"[0.6452325582504272, 0.6240962743759155]",0.95,0.63,success,refute,0
42,Antimicrobial agents are less effective due to...,SUPPORTS,[Overusing antibiotics is not the only cause a...,train,6,9,"[tfr2xbxu.00064, 8wvclg4o.00006, ytv7dwr0.0004...","[8dxv04uu.00010, bwn4bmf5.00017, pzjj2wxc.0000...","[0.9899699091911316, 0.7421004772186279, 0.677...","[0.9373342394828796, 0.8747098445892334, 0.837...",0.70,0.71,success,refute,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,There is an inverse relationship between hip f...,SUPPORTS,[CONCLUSIONS These findings support an associa...,dev,0,5,[],"[f1br2h6p.00045, 7r35ow4y.00059, 46oo3sse.0000...",[],"[0.911043107509613, 0.7290191054344177, 0.6985...",0.00,0.70,success,refute,0
676,Thigh-length graduated compression stockings (...,SUPPORTS,[INTERPRETATION These data do not lend support...,dev,3,4,"[37zrbye6, ie9dn724.00008, 95yiliim]","[nuadau49, 2uzai25k, 2n9f82xs, 6vxtpn90]","[0.8855438232421875, 0.7021892070770264, 0.672...","[0.6754951477050781, 0.637149453163147, 0.5785...",0.75,0.61,success,refute,0
677,Tirasemtiv has no effect on fast-twitch muscle.,REFUTES,[Binding selectively to the fast-skeletal-trop...,dev,2,4,"[kxfsusgo, 2v4w7ail]","[jns9aaez, h2p8id96, fkqkbail, 16skmh6b.00030]","[0.8443883061408997, 0.7982557415962219]","[0.9675067067146301, 0.9525337815284729, 0.847...",0.82,0.87,success,refute,0
679,Transplanted human glial progenitor cells are ...,REFUTES,[Long-term potentiation (LTP) was sharply enha...,dev,0,1,[],[dkze9ees.00047],[],[0.548649787902832],0.00,0.55,success,refute,0


In [30]:
le_name_mapping = dict(zip(le_ground.classes_, le_ground.transform(le_ground.classes_)))
print(le_name_mapping)

{'refute': 0}


In [31]:
tpl_claim_label_rationale = list(df_claim_evid_label[['org_claim', 'label_enc', 'label', 
                                                      'ground_list_rationales', 'source']].itertuples(index=False, name=None))

In [32]:
tpl_claim_label_rationale

[('76-85% of people with severe mental disorder receive no treatment in low and middle income countries.',
  0,
  'refute',
  ['Although disorder severity was correlated with probability of treatment in almost all countries, 35.5% to 50.3% of serious cases in developed countries and 76.3% to 85.4% in less-developed countries received no treatment in the 12 months before the interview.'],
  'train'),
 ('ART has no effect on the infectiveness of HIV-positive people.',
  0,
  'refute',
  ['CONCLUSIONS Mathematical models evaluating the impact of ART vary substantially in structure, complexity, and parameter choices, but all suggest that ART, at high levels of access and with high adherence, has the potential to substantially reduce new HIV infections.'],
  'train'),
 ('Ambulatory blood pressure monitoring is inaccurate at diagnosing hypertension.',
  0,
  'refute',
  ['Ambulatory monitoring before the start of lifelong drug treatment might lead to more appropriate targeting of treatment, 

In [33]:
#ds_claim_label  = textattack.datasets.Dataset(tpl_claim_label_rationale, input_columns=(['claim']))
#ds_claim_label  = textattack.datasets.Dataset(tpl_claim_label_rationale, input_columns=(['text']), label_names = ['no_result', 'equal', 'support', 'refute'])
#ds_claim_label  = textattack.datasets.Dataset(tpl_claim_label_rationale, input_columns=(['text']), label_names = ['support', 'refute'])
ds_claim_label  = textattack.datasets.Dataset(tpl_claim_label_rationale, input_columns=(['text']), label_names = ['refute', 'support'])

## TextAttack Custom attack


In [34]:
from textattack.models.wrappers import ModelWrapper

In [35]:
class ScifactModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model
    def __call__(self, list_claim_pair):
        #print('list_claim_pair : ', list_claim_pair)
        list_all_probs = []
        logging.info(">>> \n\nList >>> ")
        logging.info(list_claim_pair)
        for cur_claim in list_claim_pair:
            prob_dist_cur = get_prob_score_claim(cur_claim)
            list_all_probs.append(prob_dist_cur)
#         prob_dist_org = get_prob_score_claim(list_claim_pair[0])
#         prob_dist_org = torch.tensor([prob_dist_org])
#         prob_dist_gen = get_prob_score_claim(list_claim_pair[1])
#         prob_dist_gen = torch.tensor(prob_dist_gen)        
        
#         final_pred = torch.stack((prob_dist_org, prob_dist_gen), dim = 0)
        
#         return final_pred
        prob_dist_all = np.array(list_all_probs)
        #print('prob_dist_all : ', prob_dist_all)
        return prob_dist_all

In [36]:
ds_claim_label[0]

(OrderedDict([('text',
               '76-85% of people with severe mental disorder receive no treatment in low and middle income countries.')]),
 0)

In [37]:
perturb_results_txt_fooler = []

## TextFooler

In [38]:
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019

In [39]:
class_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']
#attack_args_fooler = AttackArgs(num_examples=-1)
attack_args_fooler = AttackArgs(num_examples=-1, log_to_csv="../../logs/log_textattack_results_textfooler_refute.csv", csv_coloring_style="html")
model_wrapper_txt_fooler = ScifactModelWrapper(class_model)
attack_txt_fooler = TextFoolerJin2019.build(model_wrapper_txt_fooler)
attacker_txt_fooler = Attacker(attack_txt_fooler, ds_claim_label, attack_args_fooler)
attack_results_txt_fooler = attacker_txt_fooler.attack_dataset()

textattack: Unknown if model of class <class 'str'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../../logs/log_textattack_results_refute.csv
  0%|          | 0/118 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/118 [07:57<15:30:45, 477.31s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Refute (100%)]] --> [[Support (100%)]]

76-85% of people with severe mental disorder [[receive]] no treatment in low and middle income countries.

76-85% of people with severe mental disorder [[receiving]] no treatment in low and middle income countries.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/118 [10:24<10:03:55, 312.38s/it]

...

--------------------------------------------- Result 116 ---------------------------------------------
[[Refute (100%)]] --> [[[FAILED]]]

Tirasemtiv has no effect on fast-twitch muscle.




[Succeeded / Failed / Skipped / Total] 84 / 33 / 0 / 117:  99%|█████████▉| 117/118 [18:11:28<09:19, 559.73s/it]

--------------------------------------------- Result 117 ---------------------------------------------
[[Refute (100%)]] --> [[Support (100%)]]

Transplanted human glial progenitor cells are [[incapable]] of forming a neural network with host animals' neurons.

Transplanted human glial progenitor cells are [[vulnerable]] of forming a neural network with host animals' neurons.




[Succeeded / Failed / Skipped / Total] 85 / 33 / 0 / 118: 100%|██████████| 118/118 [18:15:16<00:00, 556.92s/it]

--------------------------------------------- Result 118 ---------------------------------------------
[[Refute (100%)]] --> [[Support (100%)]]

Ultrasound guidance significantly [[raises]] the number of traumatic procedures when attempting needle insertion.

Ultrasound guidance significantly [[begs]] the number of traumatic procedures when attempting needle insertion.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 85     |
| Number of failed attacks:     | 33     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 27.97% |
| Attack success rate:          | 72.03% |
| Average perturbed word %:     | 10.18% |
| Average num. words per input: | 11.86  |
| Avg num queries:              | 100.85 |
+-------------------------------+--------+


In [40]:
for cur_result in attack_results_txt_fooler:
    print()
    print()
    print(cur_result.__str__(color_method='ansi'))



Refute (100%) --> Support (100%)

76-85% of people with severe mental disorder receive no treatment in low and middle income countries.

76-85% of people with severe mental disorder receiving no treatment in low and middle income countries.


Refute (100%) --> Support (100%)

ART has no effect on the infectiveness of HIV-positive people.

KIND has no effect on the infectiveness of HIV-positive people.


Refute (100%) --> [FAILED]

Ambulatory blood pressure monitoring is inaccurate at diagnosing hypertension.


Refute (100%) --> Support (100%)

Antidepressants increase the severity of migraines.

Ambien increase the severity of migraines.


Refute (100%) --> Support (100%)

Antimicrobial agents are less effective due to the pressure of antimicrobial usage.

Antimicrobial agents are less effective due to the insist of antimicrobial usage.


Refute (100%) --> Support (100%)

Antimicrobial agents are more effective due to the pressure of antimicrobial usage.

Antimicrobial retainers are 

In [41]:
logger = attacker_txt_fooler.attack_log_manager.loggers[0]
successes = logger.df
display(HTML(successes[['original_text', 'perturbed_text', 'result_type']].to_html(escape=False)))

,original_text,perturbed_text,result_type
0,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with severe mental disorder receiving no treatment in low and middle income countries.,Successful
1,ART has no effect on the infectiveness of HIV-positive people.,KIND has no effect on the infectiveness of HIV-positive people.,Successful
2,Ambulatory blood pressure monitoring is inaccurate at diagnosing hypertension.,Ambulatory blood pressure monitoring is inaccurate at diagnosing hypertension.,Failed
3,Antidepressants increase the severity of migraines.,Ambien increase the severity of migraines.,Successful
4,Antimicrobial agents are less effective due to the pressure of antimicrobial usage.,Antimicrobial agents are less effective due to the insist of antimicrobial usage.,Successful
5,Antimicrobial agents are more effective due to the pressure of antimicrobial usage.,Antimicrobial retainers are more effective due to the pressure of antimicrobial usage.,Successful
6,Antiretroviral therapy increases rates of tuberculosis across a broad range of CD4 strata.,Antiretroviral therapy augment rates of tuberculosis across a broad range of CD4 strata.,Successful
7,Autophagy deficiency in the liver increases vulnerability to insulin resistance.,Autophagy deficiency in the liver growing vulnerability to insulin resistance.,Successful
8,BCL-2 activation antagonizes the apoptotic effects of c-Myc.,BCL-2 activated antagonizes the apoptotic effects of c-Myc.,Successful
9,BCL-2 promotes the apoptotic effects of c-Myc.,BCL-2 promotes the apoptotic afflict of c-Myc.,Successful


In [42]:
perturb_results_txt_fooler.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'TextFoolerJin2019') for cur_result in attack_results_txt_fooler] )

In [43]:
df_txt_fooler = pd.DataFrame(perturb_results_txt_fooler, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [44]:
df_txt_fooler

,org_claim,gen_claim,num_of_query,attack_result,attack_method
0,76-85% of people with severe mental disorder r...,76-85% of people with severe mental disorder r...,62,SuccessfulAttackResult,TextFoolerJin2019
1,ART has no effect on the infectiveness of HIV-...,KIND has no effect on the infectiveness of HIV...,22,SuccessfulAttackResult,TextFoolerJin2019
2,Ambulatory blood pressure monitoring is inaccu...,Ambulatory blood pressure monitoring is inaccu...,179,FailedAttackResult,TextFoolerJin2019
3,Antidepressants increase the severity of migra...,Ambien increase the severity of migraines.,30,SuccessfulAttackResult,TextFoolerJin2019
4,Antimicrobial agents are less effective due to...,Antimicrobial agents are less effective due to...,41,SuccessfulAttackResult,TextFoolerJin2019
...,...,...,...,...,...
113,There is an inverse relationship between hip f...,There is an invest relationship between hip fr...,28,SuccessfulAttackResult,TextFoolerJin2019
114,Thigh-length graduated compression stockings (...,Thigh-length graduated compression stockings (...,72,SuccessfulAttackResult,TextFoolerJin2019
115,Tirasemtiv has no effect on fast-twitch muscle.,Tirasemtiv has no effect on fast-twitch muscle.,73,FailedAttackResult,TextFoolerJin2019
116,Transplanted human glial progenitor cells are ...,Transplanted human glial progenitor cells are ...,71,SuccessfulAttackResult,TextFoolerJin2019


In [45]:
df_txt_fooler.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_textfooler_jin_2019_ref_to_sup_2_opt.pkl')
df_txt_fooler.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_textfooler_jin_2019_ref_to_sup_2_opt.csv')

## CheckList

In [38]:
logging.info(">>> \n\\n CheckList >>> ")

In [39]:
from textattack.attack_recipes.checklist_ribeiro_2020 import CheckList2020

In [40]:
class_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']

attack_args_cklist = AttackArgs(num_examples=-1, log_to_csv="../../logs/checklist_log_results_refute.csv", csv_coloring_style="html")
model_wrapper_cklist = ScifactModelWrapper(class_model)

attack_checklist = CheckList2020.build(model_wrapper_cklist)

attacker_checklist = Attacker(attack_checklist, ds_claim_label, attack_args_cklist)
attack_results_checklist = attacker_checklist.attack_dataset()

textattack: Unknown if model of class <class 'str'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../../logs/checklist_log_results_refute.csv
  0%|          | 0/118 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapExtend
    (1): WordSwapContract
    (2): WordSwapChangeName
    (3): WordSwapChangeNumber
    (4): WordSwapChangeLocation
    )
  (constraints): 
    (0): RepeatModification
  (is_black_box):  True
) 



/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


2022-01-24 15:50:52,407 --------------------------------------------------------------------------------
2022-01-24 15:50:52,408 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2022-01-24 15:50:52,409  - The most current version of the model is automatically downloaded from there.
2022-01-24 15:50:52,409  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2022-01-24 15:50:52,410 --------------------------------------------------------------------------------
2022-01-24 15:50:52,790 loading file /home/qudratealahyratu/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/118 [00:10<20:26, 10.49s/it]

...

--------------------------------------------- Result 116 ---------------------------------------------
[[Refute (100%)]] --> [[[FAILED]]]

Tirasemtiv has no effect on fast-twitch muscle.




[Succeeded / Failed / Skipped / Total] 1 / 116 / 0 / 117:  99%|█████████▉| 117/118 [17:56<00:09,  9.20s/it]

--------------------------------------------- Result 117 ---------------------------------------------
[[Refute (100%)]] --> [[[FAILED]]]

Transplanted human glial progenitor cells are incapable of forming a neural network with host animals' neurons.




[Succeeded / Failed / Skipped / Total] 1 / 117 / 0 / 118: 100%|██████████| 118/118 [18:00<00:00,  9.15s/it]

--------------------------------------------- Result 118 ---------------------------------------------
[[Refute (100%)]] --> [[[FAILED]]]

Ultrasound guidance significantly raises the number of traumatic procedures when attempting needle insertion.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 117    |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 99.15% |
| Attack success rate:          | 0.85%  |
| Average perturbed word %:     | 11.11% |
| Average num. words per input: | 11.86  |
| Avg num queries:              | 1.71   |
+-------------------------------+--------+


In [41]:
perturb_results_cklist = []

In [42]:
perturb_results_cklist.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'checklist_ribeiro_2020') for cur_result in attack_results_checklist] )

In [43]:
df_cklist= pd.DataFrame(perturb_results_cklist, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [44]:
df_cklist.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_checklist_ribeiro_2020_ref_to_sup_2_opt.pkl')
df_cklist.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_checklist_ribeiro_2020_ref_to_sup_2_opt.csv')

### BAE (BAE: BERT-Based Adversarial Examples)

In [37]:
logging.info(">>> \n\nBAE >>> ")

In [38]:
from textattack.attack_recipes.bae_garg_2019 import BAEGarg2019

In [ ]:
class_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']

attack_args_bae = AttackArgs(num_examples=-1, log_to_csv="../../logs/log_textattack_bae_refute.csv", csv_coloring_style="html")
model_wrapper_bae = ScifactModelWrapper(class_model)
attack_bae = BAEGarg2019.build(model_wrapper_bae)

attacker_bae = Attacker(attack_bae, ds_claim_label, attack_args_bae)
attack_results_bae = attacker_bae.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'str'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../../logs/log_textattack_bae_refute.csv
  0%|          | 0/118 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
perturb_results_bae = []

In [ ]:
perturb_results_bae.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'bae_garg_2019') for cur_result in attack_results_bae] )

In [ ]:
df_bae= pd.DataFrame(perturb_results_bae, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [ ]:
df_bae.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_bae_garg_2019_ref_to_sup_2opt.pkl')
df_bae.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifact_bae_garg_2019_ref_to_sup_2opt.csv')

## BERT-ATTACK 

Applied the modification of:
https://github.com/QData/TextAttack/issues/586
> max_candidates=4

> total query = 4^4 = 256 query budget

In [ ]:
from textattack.attack_recipes.bert_attack_li_2020 import BERTAttackLi2020

In [ ]:
#model_btk = transformers.AutoModelForSequenceClassification.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])
#tokenizer_btk = transformers.AutoTokenizer.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])

#model_wrapper_btk = HuggingFaceModelWrapper(model_btk, tokenizer_btk)

In [ ]:
class_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']
#attack_args_fooler = AttackArgs(num_examples=-1)
attack_args_ber_attack = AttackArgs(num_examples=-1, log_to_csv="../../logs/bert_attack_log_results.csv", csv_coloring_style="html")
model_wrapper_txt_ber_attack = ScifactModelWrapper(class_model)
attack_txt_ber_attack = BERTAttackLi2020.build(model_wrapper_txt_ber_attack)
attacker_txt_ber_attack = Attacker(attack_txt_ber_attack, ds_claim_label, attack_args_ber_attack)
attack_results_txt_ber_attack = attacker_txt_ber_attack.attack_dataset()

In [ ]:
perturb_results_btk = []

In [ ]:
perturb_results_btk.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'bert_attack_li_2020') for cur_result in attack_results_txt_ber_attack] )

In [ ]:
df_btk= pd.DataFrame(perturb_results_btk, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [ ]:
df_btk.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifacti_as_black_box_model/bert_attack_li_2020_sup_to_ref_2_output.pkl')
df_btk.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/scifact_as_black_box_model_2_output/scifacti_as_black_box_model/bert_attack_li_2020_sup_to_ref_2_output.csv')